# Assignment 4: Submission Format

This notebook shows you how you should format your function in order to return your playlist of 10 songs, along with any other information you want to return.

In [102]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

# you need this for namedtuple
from collections import namedtuple
from string import  punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

# makes printing a dict look nice
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [103]:
songs_master_df = pd.read_json('MasterSongList.json')

In [104]:
songs_master_df.loc[:,'genres'] = songs_master_df['genres'].apply(''.join)
def consolidateGenre(genre):
    if(len(genre) > 0):
        return genre.split(':')[0]
    else:
        return genre
songs_master_df.loc[:,'genres'] = songs_master_df['genres'].apply(consolidateGenre)
genres_moods_df = pd.concat([songs_master_df['genres'], songs_master_df['moods'], songs_master_df['artist'], songs_master_df['name']], axis=1)

In [105]:
genres_moods_dfgenres_m  = genres_moods_df.join(pd.DataFrame(mlb.fit_transform(genres_moods_df['moods']), columns=mlb.classes_, index=genres_moods_df.index))
# genres_moods_df.drop('moods', inplace=True, axis=1)
genres_moods_df.dropna(how='any', inplace=True)
genres_moods_df.head()

,genres,moods,artist,name
0,pop,"[energetic, motivational]",PSY,Gangnam Style (강남스타일)
1,pop,[happy],OneRepublic,Counting Stars
2,,"[happy, celebratory, rowdy]",LMFAO,Party Rock Anthem
3,dance,"[happy, energetic, celebratory]",PSY,Gentleman
4,reggaeton,[energetic],Jennifer Lopez,On The Floor


In [106]:
mlb = MultiLabelBinarizer()

In [107]:
translator = str.maketrans('','', punctuation)
stemmer = SnowballStemmer('english')

In [108]:
# Create a namedtuple called 'Song'
Song = namedtuple("Song", ["artist", "title", "info"])
# Create a 'mood'
Mood = namedtuple("Mood", ["description", "probability"])

In [109]:
def clean_text(raw_text):
    clean_words = []
    raw_text.lower()
    raw_text = raw_text.translate(translator)
    split_words = raw_text.split()
    
    for word in split_words:
        if word not in ENGLISH_STOP_WORDS:
            word = stemmer.stem(word)
            clean_words.append(word)
            
    return ' '.join(clean_words)

In [110]:
from sklearn.metrics import jaccard_similarity_score
def check(raw_text, raw_text2):
    clean_words = []
    clean_words.append(jaccard_similarity_score(raw_text,raw_text2))
    return clean_words

print(check(['o'], ['1']))

[0.0]


In [141]:
# You should submit code that basically looks like this
def recommend_similar_songs(audio_features, lyrics_features):
    # ALL YOUR CODE
    # You can also include more information if you like. For example, if you decide to include Top 3 Moods & Genre:

    # I'm choosing to also include probability, so I'm making Mood a namedtuple as well
#     Mood = namedtuple("Mood", ["description", "probability"])

#     top_mood_1 = Mood(description='Happy', probability=0.75)
#     top_mood_2 = Mood(description='Energetic', probability=0.71)
#     top_mood_3 = Mood(description='Celebratory', probability=0.68)

#     `
#     final_result_dictionary['genre'] = 'Jazz'

#     #pp.pprint(final_result_dictionary)



#     # Create a namedtuple called 'Song'
#     Song = namedtuple("Song", ["artist", "title", "info"])

#     song_1 = Song(artist="kanye west", title='i am a god', info = final_result_dictionary)
#     song_2 = Song(artist="linkin park", title="crawling",  info = final_result_dictionary)
#     # etc.

#     # Return your results as a dict containing a key called 'playlist' , which contains a list of the song tuples.
#     final_result_dic = dict(playlist=[song_1, song_2]) # this example uses only 2 songs but you need to return 10 :)
#     pp.pprint(final_result_dic)
#     tf_idf_vect = TfidfVectorizer()

#     tf_idf = tf_idf_vect.fit_transform([lyrics_clean])
    
#     print(tf_idf)

    from sklearn.metrics import jaccard_similarity_score
    df_moods = songs_master_df['moods'].str.join(',').str.get_dummies(sep=',')
    df_genres_moods = pd.concat((songs_master_df['genres'], df_moods), axis=1)

    moods_features_predict  = pickle.load(open('moods_lyrics_tfidf.pickle', 'rb'))
    genres_features_predict  = pickle.load(open('genres_features.pickle', 'rb'))  
#     print(genres_features_predict)
#     print(moods_features_predict)
    genres = genres_features_predict.predict(np.array(audio_features).reshape((1, -1)))
    
    y = mlb.fit_transform(songs_master_df['moods'])
    lyrics_clean = clean_text(lyrics_features)
    moods = moods_features_predict.predict([lyrics_clean])
    moods = mlb.inverse_transform(moods)
    
#     songs_master_df['genres'] = songs_master_df['genres'].apply(''.join)
#     songs_master_df['moods'] = songs_master_df['moods'].apply(', '.join

    print ("genres predict : " + genres)
    print ('mood predict: ' + str(moods))
    
    
    final_genres_moods_df = genres_moods_df.loc[(genres_moods_df['genres']) == genres[0]].copy()
#     print(final_genres_moods_df.head(10))
    final_genres_moods_df = final_genres_moods_df.drop(['genres', 'artist', 'name'], axis=1)
    print(final_genres_moods_df)
    for index, row in final_genres_moods_df.iterrows():
        final_genres_moods_df['scsore'] = jaccard_similarity_score(row, moods[0])
        
    final_genres_moods_df.sort_values(by=['score'])
    print(final_genres_moods_df.head(10))
    
    
    TopMoods1 = Mood(description='happy', probability=0.5)
    TopMoods2 = Mood(description='sad', probability=0.3)
    TopMoods3 = Mood(description='fun', probability=0.1)

    top_moods_dictionary = [top_mood_1, top_mood_2, top_mood_3]

    song_0 = Song(artist="kanye west", title='i am a god', info = top_moods_dictionary)
    song_1 = Song(artist="thanh", title='beatbox show', info = top_moods_dictionary)

    final_songs_dictionary = dict(playlist=[song_0, song_1])

    final_songs_dictionary
    
    return final_songs_dictionary    

In [142]:
recommend_similar_songs(songs_master_df['audio_features'][1], 'get hight tonight')

['genres predict : rock']
mood predict: [('cocky', 'energetic', 'motivational', 'visceral')]
                                         moods
25                                    [trippy]
47                         [aggressive, rowdy]
54                                    [trippy]
60                             [cocky, trashy]
70                                     [sweet]
86                                     [sweet]
92                             [cocky, trashy]
106                                      [sad]
122                  [energetic, motivational]
157                                   [trippy]
168                           [sad, nocturnal]
182                               [aggressive]
199                                   [mellow]
200                                    [sweet]
201                               [aggressive]
210                             [motivational]
216                                   [angsty]
225                                   [mellow]
226           

/Volumes/Data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

---